In [1]:
import pandas as pd
import os
os.chdir('/home/jovyan/shared/2020_06_10_bad_reviewer/')

## Reviews

In [2]:
reviews = pd.read_csv('data/coursera_tables/peer_reviews.csv',
                      usecols=['peer_review_id',
                               'peer_submission_id',
                               'uva_peer_assignments_user_id',
                               'peer_review_created_ts'])

In [3]:
schema = pd.read_csv('data/coursera_tables/peer_assignment_review_schema_parts.csv')
schema_options = pd.read_csv('data/coursera_tables/peer_assignment_review_schema_part_options.csv')
# display(schema)
# display(schema_options)

In [4]:
written_responses = pd.read_csv('data/coursera_tables/peer_review_part_free_responses.csv', escapechar='\\', on_bad_lines='warn')
score_responses = pd.read_csv('data/coursera_tables/peer_review_part_choices.csv')
# display(written_responses)
# display(score_responses)

In [5]:
human_readable_text_responses = written_responses.merge(schema)
'''
subtract 1 from each schema part order, so the text response schema_part_order has the same value as the score_response before it
So if "Challenge (score)" has schema_part_order 0 and "Challenge Feedback (ungraded)" has schema_part_order 1,
We can align these dataviews and get a single row for Challenge, both score and feedback.

'''
human_readable_text_responses.peer_assignment_review_schema_part_order = human_readable_text_responses.peer_assignment_review_schema_part_order -1

In [6]:
reviews_df = (score_responses # what each reviewer scored each assignment, by component on the analytic scale
              .merge(schema_options) # what information was presented to the reviewer (what was the prompt?)
              .merge(schema)[[
                  'peer_review_id',
                  'peer_assignment_review_schema_part_prompt',
                  'peer_assignment_review_schema_part_order',
                  'peer_assignment_review_schema_part_option_score']]
              .merge(human_readable_text_responses,
                     on=['peer_review_id', 'peer_assignment_review_schema_part_order'],
                     how='outer',
                     suffixes=['_score', '_free_response'],
                     sort=True) # The written responses to each component of the rubric AND the prompts
              .merge(reviews, on=['peer_review_id'], how='outer')#[:-6] # the last 6 have no scores, but I should leave them as NA values to be dealt with later. This is not the time for cleaning. Keep the data raw until the analysis.
             )[['peer_review_id',
                'uva_peer_assignments_user_id',
                'peer_review_created_ts',
                'peer_submission_id',
                'peer_assignment_review_schema_part_prompt_score',
                'peer_assignment_review_schema_part_option_score',
                'peer_assignment_review_schema_part_prompt_free_response',
                'peer_review_part_free_response_text']]

display(reviews_df)
# reviews_df.to_csv('bin/reviews_df.csv')

,peer_review_id,uva_peer_assignments_user_id,peer_review_created_ts,peer_submission_id,peer_assignment_review_schema_part_prompt_score,peer_assignment_review_schema_part_option_score,peer_assignment_review_schema_part_prompt_free_response,peer_review_part_free_response_text
0,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Challenge (score),3.0,Feedback (ungraded):The submission’s challenge...,search for higher ground
1,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Selection (score),3.0,Feedback (ungraded):The submission’s tool sele...,removing barriers
2,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Application (score),2.0,Feedback (ungraded):The submission’s tool appl...,understanding it
3,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Insight (score),3.0,Feedback (ungraded):The submission’s insight d...,drilling down to the essence
4,--1K-8NvEeqUARLn1lBFnw,b3a960970d72a6ce0b00c0bdfbccd943b9b64b6a,2020-07-11 12:13:52.365,tyljI8NkEeql1A5WxvNUrQ,Approach (score),3.0,Feedback (ungraded):The submission’s approach ...,increasing speed of learning
...,...,...,...,...,...,...,...,...
1084658,ARYplXxXEeq-IBL6cGi2Cw,a6131cda10617880caeff202f24b9de41fcfa722,2020-04-12 00:46:11.045,C3DnznwuEeqEUA6YVkoxuQ,NaN,NaN,NaN,NaN
1084659,GfkaLiyFEeu4Zgo0ZCoTOQ,8284087aea601fc9b56dd29a473ecc7c65819a8b,2020-11-22 05:39:34.297,bx0OtcZQEeq3ahK57vJ81Q,NaN,NaN,NaN,NaN
1084660,KcruGXxZEeqEUA6YVkoxuQ,ff41433c3431f10ab61b6b89506d88194f8be736,2020-04-12 01:01:38.332,C3DnznwuEeqEUA6YVkoxuQ,NaN,NaN,NaN,NaN
1084661,h0R7LNUwEeqdnQrjXnH35w,49163e3c086ea334b4b145a8a3e61738b957d84b,2020-08-03 02:24:59.278,fOFFBdTSEeqwjRINUYbZPQ,NaN,NaN,NaN,NaN


## Submissions

cleared_submissions.csv is a subset of the Coursera submissions.csv table.  
It relies on criteria defined in a set of SQL queries

These queries are documented in [submission_queries.md](http://as-alesl-nlp.dyn.gsu.edu:31151/hub/user-redirect/lab/tree/shared/2020_06_10_bad_reviewer/src/submission_queries.md)

In [7]:
submissions = pd.read_csv('data/coursera_tables/peer_submissions.csv', escapechar='\\', on_bad_lines='warn')
print(f'Some reviews are associated with submission IDs that do not exist in the submissions table provided by Coursera')
submissions_represented = reviews_df.peer_submission_id.isin(submissions.peer_submission_id).value_counts()
print(f'{submissions_represented[1]:,} submission ids are reflected in the submissions table; {submissions_represented[0]:,} are not.')

Some reviews are associated with submission IDs that do not exist in the submissions table provided by Coursera
1,083,641 submission ids are reflected in the submissions table; 1,022 are not.


### Which submissions were cleared in the SQL database?

In [8]:
cleared_submissions = pd.read_csv('bin/cleared_submissions.csv', escapechar='\\', on_bad_lines='warn')
submissions['is_cleared'] = submissions.peer_submission_id.isin(cleared_submissions.peer_submission_id)

print(f'Number of unique users who submitted: {submissions.uva_peer_assignments_user_id.nunique():,}')
print(f'Number of unique users who were cleared: {submissions[submissions.is_cleared].uva_peer_assignments_user_id.nunique():,}')

excluded = submissions[~submissions.is_cleared].peer_submission_id.nunique()
total = submissions.peer_submission_id.nunique()
print(f'{excluded:,} submissions, of {total:,} submissions in total, would be excluded from reviews_df if we only considered cleared submissions.')

Number of unique users who submitted: 43,167
Number of unique users who were cleared: 36,623
184,420 submissions, of 221,043 submissions in total, would be excluded from reviews_df if we only considered cleared submissions.


### How many times was each submission skipped by peer reviewers?

In [10]:
submission_skips = pd.read_csv('data/coursera_tables/peer_skips.csv', escapechar='\\', on_bad_lines='warn')

submissions['num_skips'] = (submission_skips
                            .value_counts(subset='peer_submission_id')
                            .reindex(index=submissions.peer_submission_id)
                            .fillna(0)
                            .astype(int)
                            .values
                              )

### Add the text of each parsed submission.
These are the only submissions we actually care about.

Also mark which submissions were written by users who posted in the discussion forum.

In [11]:
texts = pd.read_json('data/cleared_txts.jsonl', lines=True, orient='records')
texts['id'] = texts['id'].str.split('_').str[1]

labelled_txts = pd.read_json('data/labelled_subs_with_posts.jsonl', orient='records', lines=True)
labelled_txts['id'] = labelled_txts['id'].str.split('_').str[1]
texts['has_post'] = texts.id.isin(labelled_txts.id)

submissions_df = (texts
                  .drop(['label'], axis=1)
                  .rename(columns={'id': 'uva_peer_assignments_user_id'})
                  .merge(submissions[submissions.is_cleared],
                         on=['uva_peer_assignments_user_id'],
                         how='left')
                 )[['peer_submission_id',
                    'uva_peer_assignments_user_id',
                    'text',
                    'peer_submission_score',
                    'num_skips',
                    'has_post',
                    'is_cleared']]

print(f'{submissions_df[submissions_df.peer_submission_id.isna()].shape[0]} text files have peer_assignments_user_id values that are not in the cleared_submissions table.')
submissions_df = submissions_df.dropna(subset=['peer_submission_score'])

print(f'{submissions_df.shape[0]:,} examples, and {submissions_df.has_post.sum():,} of those samples come from users with posts.')

display(submissions_df)
# submissions_df.to_csv('bin/submissions_df.csv')

438 text files have peer_assignments_user_id values that are not in the cleared_submissions table.
27,909 examples, and 3,216 of those samples come from users with posts.


,peer_submission_id,uva_peer_assignments_user_id,text,peer_submission_score,num_skips,has_post,is_cleared
0,LDdVsMGhEemiSwrCiJNKAg,0000e5af02da0c7575b3ebd346b55b29f959e90f,Reflection – visualization\n\nChallenge &...,15.0,0.0,False,True
1,x6_F_AF4EeqBNgqlMZTYEw,0002c0f31f5c8456bd360dfcd089a64f444e2de0,Assignment :\n\n1. Challenge: Describe your ch...,17.0,0.0,False,True
2,5IVymkAeEeyWsA72VEvfZw,00030b378ea62d60a177113b7854eb26cc29e1a9,VISUALIZATION\n\nChallenge I am part of the...,14.0,0.0,False,True
3,LCRPeX7LEeubEg4S-1QqIw,000458f7d47a0b6414f9146258829170ae3ed6a9,The main item is usually the focal point of th...,18.0,0.0,False,True
4,NGPJ3skwEeuLjgr2Ka7enw,00069909160c6bcd9836cdb23e35b1fdaf56d0c3,Storytelling is when narratives people tell ab...,18.0,0.0,False,True
...,...,...,...,...,...,...,...
28342,LgcsFcfEEemRWg4GTNs16g,fff43ee22d85efffada33d73f953c0d814d90290,Peter Vogt - Design Thinking Reflection - Lear...,16.0,0.0,False,True
28343,CrXeU6N6Eeq5qg5t3Q_jkQ,fff56a1858c62540bd76bad23db07a2fbfa963fe,STORYTELLING AS A TOOL FOR STRATEGIC CHANGES\n...,17.0,0.0,False,True
28344,vXlp8YvQEemLCQ6udTRPUg,fff7364558963fb9fa218f2fa08d5d2767992207,Junio 2019 Storytelling Jose Vergara\n\nChalle...,17.0,0.0,True,True
28345,qZ-lqiCnEemqyApQiwO8gA,fffa6add42713b8b1e2a5158616f780997bbda49,Reflexion - Mind Mapping\n\nChallenge & Select...,12.0,0.0,False,True


## Starting Correlation

Please keep in mind that the peer_submission_score here is the one provided by Coursera. This is not the simple average of the peer review scores, so the scores we calculate will diverge from this if we use a simple average. See the section **Why aren't peer_submission_scores the average of the review scores??**

In [159]:
submissions_df['word_count'] = submissions_df.text.str.split(' ').str.len()
submissions_df.corr()

,peer_submission_score,num_skips,has_post,word_count
peer_submission_score,1.000000,-0.019220,0.021417,0.111723
num_skips,-0.019220,1.000000,0.083510,0.010441
has_post,0.021417,0.083510,1.000000,0.031464
word_count,0.111723,0.010441,0.031464,1.000000


# Test Bed

Commands that I don't want to delete because they took a while to write, but don't really do anything at this point...

In [23]:
labelled_txts = pd.read_json('data/labelled_txts/all_labelled_submissions_6293.jsonl', orient='records', lines=True)
labelled_txts['id'] = labelled_txts['id'].str.split('_').str[1]
# display(labelled_txts)

# The following is because some labelled submissions were labelled/annotated before we settled on our final criteria
# They can safely be ignored.

print('total labelled submissions is', labelled_txts.shape[0])
print('num labelled submissions reflected in Coursera submissions is', labelled_txts[labelled_txts.id.isin(submissions.uva_peer_assignments_user_id)].shape[0])
print('num labelled submissions reflected in our submissions_df is', labelled_txts[labelled_txts.id.isin(submissions_df.uva_peer_assignments_user_id)].shape[0])

total labelled submissions is 6293
num labelled submissions reflected in submissions_df is 6230
num labelled submissions reflected in Coursera submissions is 6293


In [62]:
# The texts we care most about our the submissions from users with posts
# These ones are consistent.
labelled_txts = pd.read_json('data/labelled_txts/all_with_posts.jsonl', orient='records', lines=True)
labelled_txts['id'] = labelled_txts['id'].str.split('_').str[1]

print('total labelled submissions is', labelled_txts.shape[0])
print('num labelled submissions reflected in data_view is', labelled_txts[labelled_txts.id.isin(human_readable_scores_and_responses_with_reviews_and_author.uva_peer_assignments_user_id_author)].shape[0])
print('num labelled submissions reflected in cleared_submissions is', labelled_txts[labelled_txts.id.isin(submissions.uva_peer_assignments_user_id)].shape[0])

total labelled submissions is 3216
num labelled submissions reflected in data_view is 3216
num labelled submissions reflected in cleared_submissions is 3216


In [76]:
# While we are here. Let's see how many duplicate values we have...

print('Unique ids', labelled_txts.id.nunique())
print('Unique texts', labelled_txts.data.nunique())
display(labelled_txts[labelled_txts.duplicated(subset=['data'], keep=False)].data.value_counts())

Unique ids 3216
Unique texts 3057


How many duplicate reviews are there? Did coursera record duplicate reviews, or did I do that somehow?

In [16]:
display(reviews[reviews.duplicated()])
display(reviews_df[reviews_df.duplicated(keep=False)])

,peer_review_id,peer_submission_id,uva_peer_assignments_user_id,peer_review_created_ts


,peer_review_id,uva_peer_assignments_user_id,peer_review_created_ts,peer_submission_id,peer_assignment_review_schema_part_prompt_score,peer_assignment_review_schema_part_option_score,peer_assignment_review_schema_part_prompt_free_response,peer_review_part_free_response_text


In [22]:
# reviews_df.iloc[755643:755650]
reviews[reviews.uva_peer_assignments_user_id == '121ade9d078639b186c58f9cd2aaced28dc6dc75']

,peer_review_id,peer_submission_id,uva_peer_assignments_user_id,peer_review_created_ts
144323,a93jYDdOEeeTXBLd9rJhog,JGwq0TccEeesQRJzrNdi7A,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-12 20:06:01.477
144324,wJFFnzgEEeeVrA7fHHMkEg,6_j1AjfhEeeTXBLd9rJhog,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-13 17:51:11.986
144325,jcvdljgBEeeVrA7fHHMkEg,7oUR7jf2EeeSURIzKx1jwA,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-13 17:28:18.316
144326,g_vdLzgGEeeSURIzKx1jwA,S6IA7Df2EeesQRJzrNdi7A,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-13 18:03:49.337
144327,g_u1gzgGEee6YQrlQL-Uag,S6IA7Df2EeesQRJzrNdi7A,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-13 18:03:49.336
144328,uoJPRzgKEee6YQrlQL-Uag,6C57_DgCEeeSURIzKx1jwA,121ade9d078639b186c58f9cd2aaced28dc6dc75,2017-05-13 18:33:58.802


### Why aren't peer_submission_scores the average of the review scores??

In [215]:
scores = pd.read_csv('data/coursera_tables/peer_submission_part_scores.csv')
scores = scores.merge(schema)

In [326]:
import numpy as np
import statistics as stats

def arbitrate(x):
    '''
    First, take the mode. If there are multiple modes, return all of them.
    [1, 3, 3] --> 3
    [1, 2, 3] --> 1, 2, 3
    
    Then, take the mean of the modes:
    3 --> 3
    1, 2, 3 --> 2
    
    This appears to be how Coursera calculates the essay grade.
    '''
    return np.mean(stats.multimode(x))

def calculate_score(sub_id, method='arbitrate_parts'):
    # Get info about the submission, direct from Coursera table
    submission_info = submissions[submissions['peer_submission_id']==sub_id]

    # Get reviews associated with the submission
    submission_reviews = reviews_df[reviews_df['peer_submission_id']==sub_id]

    # Select only reviews before/when score was made available
    if pd.notnull(submission_info.peer_submission_score_available_ts.values):
        timely_reviews = submission_reviews[submission_reviews
                                            .peer_review_created_ts.values
                                            <=
                                            submission_info
                                            .peer_submission_score_available_ts
                                            .values]
    
    # Get score recorded in Coursera table
    coursera_score = submission_info.peer_submission_score.iloc[0]
    
    # Some submissions have null for final score.
    if pd.isnull(coursera_score):
        return 'NA Score Recorded.'
    
    # Try averaging all reviews
    if method=='average_all':
        my_score = np.mean(submission_reviews
                           .groupby('peer_review_id')['peer_assignment_review_schema_part_option_score']
                           .sum())
    
    # Try arbitrating by partial scores
    elif method=='arbitrate_parts':
        my_score = sum(timely_reviews[['uva_peer_assignments_user_id',
                                           'peer_assignment_review_schema_part_prompt_score',
                                           'peer_assignment_review_schema_part_option_score']]
                       .dropna()
                       .pivot(index='uva_peer_assignments_user_id',
                              columns='peer_assignment_review_schema_part_prompt_score',
                              values='peer_assignment_review_schema_part_option_score')
                       .agg(arbitrate)
                   )

    coursera_score = submission_info.peer_submission_score.iloc[0]

    # Debug Procedures...
    if coursera_score != my_score:
        print(f'The calculated score {my_score} does not equal the recorded score {coursera_score}.')
        
        print('The partial scores recorded by Coursera:')
        display(scores[scores['peer_submission_id']==sub_id].sort_values(by=['peer_assignment_review_schema_part_prompt']))
        
        print('All associated reviews:')
        display(submission_reviews[['uva_peer_assignments_user_id',
                                    'peer_assignment_review_schema_part_prompt_score',
                                    'peer_assignment_review_schema_part_option_score']]
                .dropna()
                .pivot(index='uva_peer_assignments_user_id',
                       columns='peer_assignment_review_schema_part_prompt_score',
                       values='peer_assignment_review_schema_part_option_score'))
        
        print('Reviews where timestamp is before peer_submission_score_available_ts')
        display(timely_reviews[['uva_peer_assignments_user_id',
                                'peer_assignment_review_schema_part_prompt_score',
                                'peer_assignment_review_schema_part_option_score']]
                .dropna()
                .pivot(index='uva_peer_assignments_user_id',
                       columns='peer_assignment_review_schema_part_prompt_score',
                       values='peer_assignment_review_schema_part_option_score'))

        print('Some information about the submission:')
        display(submission_info)
    
    else:
        return 'Success!'

for submission_id in reviews_df.peer_submission_id.sample(n=20):
    print(calculate_score(submission_id))

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
NA Score Recorded.
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


In [25]:
submissions_df[submissions_df.text.str.startswith('Example Reflection – Visualization')]

,peer_submission_id,uva_peer_assignments_user_id,text,peer_submission_score,num_skips,has_post,is_cleared
15,2LfoJGirEeyZgBKRrAlULw,002edb72adc00431eec524ebdffcf0a0967455f5,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
28,759e7cEeEeqb0BIFdwj-7Q,0053fa62b14adc81df54ee305ec4ad17438d7975,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
68,jTN9gULOEeuYZBI43RQb6Q,00a9b39ee739977810c76dd7c5b01e2ef164e58a,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
87,JymwoRHSEeuGFQrtOYslJw,00d4a1dab1eb36a487498d42fc1a83766fcfa4b7,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
120,E4BHa12bEeumbAqQIoY8Bw,010dec771476398b6b24b2c0ef5fd4a66727ec40,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,True,True
...,...,...,...,...,...,...,...
28150,nkjwFTUuEeulJw4UR3t4pQ,fe088508bbdf409e09561f43bbfa26af57576821,Example Reflection – Visualization\n\nChalleng...,14.0,0.0,True,True
28167,xn7GBsXeEeq3ahK57vJ81Q,fe258f8c6a7e4dd4e138415bc45909007896944a,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
28181,CLTOHSQ7EeyIag7DkTyGZQ,fe56b3da78a85379f811a1085c0fc956dd502cc2,Example Reflection – Visualization\n\nChalleng...,17.0,0.0,False,True
28250,i0an_9dqEeqZbAp7cLQpmQ,ff18224fa1f9189d78e42cb8d9f12f7d1835562e,Example Reflection – Visualization\n\nChalleng...,18.0,0.0,False,True
